In [17]:
import pandas as pd
import os
import numpy as np
from tqdm.auto import tqdm

# ======== SETTINGS ========
data_folder = r"C:/Users/yaman/OneDrive/سطح المكتب/project1/strategys/Big Char Trada"
output_folder = os.path.join(data_folder, "C:/Users/yaman/OneDrive/سطح المكتب/project1/strategys/Big Char Trada/output_folder")
os.makedirs(output_folder, exist_ok=True)

# Strategy parameters
atr_period = 14
tp_atr_factor = 1.5
sl_atr_factor = 1.0
ema_period = 50
big_candle_factor = 1.5
volume_factor = 1.5

primary_pair = "GBP/USD"
secondary_pair = "EUR/USD"

timeframe_hierarchy = {
    'M1': 'M5',
    'M5': 'M15',
    'M15': 'M30',
    'M30': 'H1',
    'H1': 'H4',
    'H4': 'D1'
}

# ======== LOAD ALL CSV FILES ========
all_files = [os.path.join(data_folder, f) for f in os.listdir(data_folder) if f.endswith('.csv')]
data_list = []

print("Loading CSV files...")
for file in tqdm(all_files, desc="Loading CSV files"):
    filename = os.path.basename(file).upper()
    
    # Determine encoding
    if "GBPUSD" in filename:
        encoding = "utf-16"
        sep = ','  # Change to ';' if columns are semicolon separated
    else:
        encoding = "utf-8-sig"
        sep = ','

    try:
        df = pd.read_csv(file, encoding=encoding, sep=sep)
    except Exception as e:
        print(f"Error reading {filename}: {e}")
        continue
    
    # Clean column names
    df.columns = df.columns.str.strip()

    # Ensure datetime column
    if 'DateTime' not in df.columns:
        df.rename(columns={df.columns[0]: 'DateTime'}, inplace=True)
    
    df['DateTime'] = pd.to_datetime(df['DateTime'], errors='coerce')

    # Assign currency pair
    if "EURUSD" in filename:
        df['Pair'] = "EUR/USD"
    elif "GBPUSD" in filename:
        df['Pair'] = "GBP/USD"
    else:
        df['Pair'] = "Other"
    
    # Assign timeframe
    for tf in ["M1","M5","M15","M30","H1","H4","D1"]:
        if tf in filename:
            df['Timeframe'] = tf
            break
    else:
        df['Timeframe'] = "Other"
    
    data_list.append(df)

# Merge all data
data = pd.concat(data_list, ignore_index=True)
data.sort_values(['Pair','Timeframe','DateTime'], inplace=True)
data.reset_index(drop=True, inplace=True)
print("CSV files loaded and merged.")

# ======== FUNCTIONS ========
def calculate_atr(df, period):
    """Calculate ATR using exponential moving average"""
    df['High_Low'] = df['High'] - df['Low']
    df['High_PrevClose'] = abs(df['High'] - df['Close'].shift(1))
    df['Low_PrevClose'] = abs(df['Low'] - df['Close'].shift(1))
    df['True_Range'] = df[['High_Low', 'High_PrevClose', 'Low_PrevClose']].max(axis=1)
    df['ATR'] = df['True_Range'].ewm(span=period, adjust=False).mean()
    df.drop(columns=['High_Low', 'High_PrevClose', 'Low_PrevClose', 'True_Range'], inplace=True)
    return df

def check_hit(open_price, high, low, close_price, direction, tp_price, sl_price):
    """Check if trade hit TP or SL"""
    if direction == 1:  # Buy
        if open_price >= tp_price: 
            return ("Win", tp_price)
        if open_price <= sl_price: 
            return ("Loss", sl_price)
    else:  # Sell
        if open_price <= tp_price: 
            return ("Win", tp_price)
        if open_price >= sl_price: 
            return ("Loss", sl_price)

    if direction == 1:
        if low <= sl_price <= high and low <= tp_price <= high:
            return ("Loss", sl_price) if sl_price < tp_price else ("Win", tp_price) 
        elif high >= tp_price:
            return ("Win", tp_price)
        elif low <= sl_price:
            return ("Loss", sl_price)
        else:
            return (None, None)
    else:
        if low <= tp_price <= high and low <= sl_price <= high:
            return ("Win", tp_price) if tp_price < sl_price else ("Loss", sl_price)
        elif low <= tp_price:
            return ("Win", tp_price)
        elif high >= sl_price:
            return ("Loss", sl_price)
        else:
            return (None, None)

# ======== PREPARE DATA ========
processed_data_frames = []
print("Calculating technical indicators (EMA, ATR, Rolling Averages)...")
for pair in tqdm(data['Pair'].unique(), desc="Processing Pairs"):
    df_pair = data[data['Pair']==pair].copy()
    for tf in df_pair['Timeframe'].unique():
        df = df_pair[df_pair['Timeframe']==tf].copy()
        if df.empty:
            continue

        df['EMA'] = df['Close'].ewm(span=ema_period, adjust=False).mean()
        df = calculate_atr(df, atr_period)
        df['CandleSize'] = df['Close'] - df['Open']
        df['Rolling_Avg_CandleSize'] = df['CandleSize'].abs().rolling(window=ema_period, min_periods=1).mean()
        if 'Volume' in df.columns:
            df['Rolling_Avg_Volume'] = df['Volume'].rolling(window=ema_period, min_periods=1).mean()
        processed_data_frames.append(df)

data_processed = pd.concat(processed_data_frames, ignore_index=True)
data_processed.sort_values(['Pair','Timeframe','DateTime'], inplace=True)
data_processed.reset_index(drop=True, inplace=True)
print("Technical indicators calculated.")

df_primary_all_tfs = data_processed[data_processed['Pair']==primary_pair].copy()

# ======== BACKTEST ========
trades = []
df_by_tf = { (p, tf): data_processed[(data_processed['Pair']==p) & (data_processed['Timeframe']==tf)].copy()
             for p in data_processed['Pair'].unique() for tf in data_processed['Timeframe'].unique() }

print("Starting backtest...")
for pair in tqdm([primary_pair, secondary_pair], desc="Backtesting Pairs"):
    df_pair_all_tfs = data_processed[data_processed['Pair']==pair].copy()
    
    for tf in tqdm(df_pair_all_tfs['Timeframe'].unique(), desc=f"  Timeframe for {pair}", leave=False):
        df = df_by_tf.get((pair, tf))
        if df is None or df.empty:
            continue
        
        for i in tqdm(range(1, len(df)), desc=f"    Rows for {tf}", leave=False):
            row = df.iloc[i]
            prev = df.iloc[i-1]

            if abs(prev['CandleSize']) < prev['Rolling_Avg_CandleSize'] * big_candle_factor:
                continue

            direction = 1 if prev['CandleSize'] > 0 else -1

            # Higher timeframe check
            higher_tf_to_check = timeframe_hierarchy.get(tf, None)
            if higher_tf_to_check:
                df_higher = df_by_tf.get((pair, higher_tf_to_check))
                if df_higher is not None and not df_higher.empty:
                    df_higher_subset = df_higher[df_higher['DateTime'] <= row['DateTime']]
                    if not df_higher_subset.empty:
                        row_higher = df_higher_subset.iloc[-1]
                        if direction == 1 and row_higher['Close'] < row_higher['EMA']:
                            continue
                        if direction == -1 and row_higher['Close'] > row_higher['EMA']:
                            continue

            # Volume check
            if 'Volume' in row.index and 'Rolling_Avg_Volume' in row.index and not pd.isna(row['Rolling_Avg_Volume']):
                if row['Volume'] < row['Rolling_Avg_Volume'] * volume_factor:
                    continue

            # Secondary pair alignment with primary
            if pair == secondary_pair and 'Primary_CandleSize' in row.index:
                primary_direction = 1 if row['Primary_CandleSize'] > 0 else -1
                if direction != primary_direction:
                    continue

            # Entry, TP, SL
            entry_price = row['Open']
            if not pd.isna(row['ATR']):
                tp_price = entry_price + direction * (row['ATR'] * tp_atr_factor)
                sl_price = entry_price - direction * (row['ATR'] * sl_atr_factor)
            else:
                tp_price = entry_price + direction * 0.0004
                sl_price = entry_price - direction * 0.0002

            result, exit_price = check_hit(entry_price, row['High'], row['Low'], row['Close'], direction, tp_price, sl_price)
            if result is None:
                result = "Win" if direction * (row['Close'] - entry_price) > 0 else "Loss"
                exit_price = row['Close']

            trades.append({
                'Pair': pair,
                'Timeframe': tf,
                'EntryDateTime': row['DateTime'],
                'EntryPrice': entry_price,
                'TP': tp_price,
                'SL': sl_price,
                'ExitPrice': exit_price,
                'Result': result,
                'ATR_at_Entry': row['ATR'] if not pd.isna(row['ATR']) else None
            })

# ======== SAVE RESULTS ========
trades_df = pd.DataFrame(trades)
total_trades = len(trades_df)
wins = trades_df[trades_df['Result']=="Win"]
losses = trades_df[trades_df['Result']=="Loss"]

print(f"\nTotal trades: {total_trades}")
print(f"Winning trades: {len(wins)}")
print(f"Losing trades: {len(losses)}")

# Save to CSV
trades_df.to_csv(os.path.join(output_folder,"all_trades.csv"), index=False)
wins.to_csv(os.path.join(output_folder,"winning_trades.csv"), index=False)
losses.to_csv(os.path.join(output_folder,"losing_trades.csv"), index=False)
print(f"Results saved to {output_folder}")


Loading CSV files...


Loading CSV files:   0%|          | 0/14 [00:00<?, ?it/s]

CSV files loaded and merged.
Calculating technical indicators (EMA, ATR, Rolling Averages)...


Processing Pairs:   0%|          | 0/2 [00:00<?, ?it/s]

Technical indicators calculated.
Starting backtest...


...

    Rows for H4:   0%|          | 0/4388 [00:00<?, ?it/s]

    Rows for M1:   0%|          | 0/1119244 [00:00<?, ?it/s]

    Rows for M30:   0%|          | 0/35035 [00:00<?, ?it/s]

    Rows for M5:   0%|          | 0/210026 [00:00<?, ?it/s]


Total trades: 62271
Winning trades: 37554
Losing trades: 24717
Results saved to C:/Users/yaman/OneDrive/سطح المكتب/project1/strategys/Big Char Trada/output_folder
